In [1]:
# 구글 드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

# 데이터 불러오기
import pandas as pd
news = pd.read_csv("/content/drive/My Drive/DSCapstone/Newsdata_URL_TEXT.csv", index_col=0)

news.head()

Mounted at /content/drive


,URL,TITLE,DATE,ARTICLE
0,https://www.hani.co.kr/arti/society/rights/111...,"서울 1~8호선 지하철, 오늘 오전 9시부터 ‘경고 파업’",2023-11-08 22:20:00,서울지하철 1~8호선을 운영하는 서울교통공사(공사) 노동조합이 9일 아침 9시부터...
1,https://www.joongboo.com/news/articleView.html...,폭염 속 지하철 1호선 지연… 시민들 ‘짜증’,2023-08-01 00:00:00,전국 대부분 지역에 폭염경보가 내려진 1일 지하철 1호선이 연착돼 이용객들이 불편을...
2,https://imnews.imbc.com/news/2024/society/arti...,강설로 서울 지하철 운행 차질..2·5·7호선 전 구간 지연,2024-02-22 07:44:00,"밤사이 내려 얼어붙은 눈으로 서울 지하철 2호선, 5호선, 7호선의 전구간 운행이 ..."
3,https://www.news1.kr/articles/5152573,"""월요일부터 지각"" 지하철 1호선 지연 운행에 시민 불편…""노조 준법투쟁""",2023-08-28 08:26:00,철도노조가 준법투쟁에 돌입한 24일 서울역 대합실에 철도노조 태업 관련 안내문이 ...
4,https://www.chosun.com/national/transport-envi...,1호선 또...지연 운행으로 퇴근길 시민들 불편,2024-03-18 00:00:00,18일 오후 경기도 의정부시 수도권 전철 1호선 회룡역에서 서울 1호선 도봉산역 ...


In [7]:
!pip install langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.7 MB/s eta 0:00:00


In [8]:
!pip install langchain

In [4]:
import getpass
import os
# 환경변수에 OpenAI API 키 저장 (사용자 입력으로 안전하게)
os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [9]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4-turbo") # GPT-4 모델로 챗봇 초기화

from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory,ConversationSummaryMemory

from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

#템플릿 생성 - 채팅 프롬프트 템플릿
from langchain_core.prompts import ChatPromptTemplate

# prompt
prompt = ChatPromptTemplate(
messages=[
SystemMessagePromptTemplate.from_template(
        "너는 네이버 블로그의 파워블로그 작가야"
),
MessagesPlaceholder(variable_name="chat_history"),# The `variable_name` here is what must align with memory
HumanMessagePromptTemplate.from_template("{question}")
]
)

# memory
memory = ConversationSummaryMemory(llm=llm, memory_key="chat_history", return_messages=True)

# conversation
conversation = LLMChain(
llm=llm,
prompt=prompt,
verbose=True,
memory=memory
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [10]:
# 질문 입력하면 GPT 답변을 반환하는 함수 정의
def question_func(x):
  answer = conversation({"question":x})
  return answer

# 실험용으로 쓸 2개의 행만 추출()
news_test = news.head(2)

In [ ]:
# 프롬프트 템플릿 정의
prompt_template = """너는 지금부터 블로그 포스팅 전문 콘텐츠 마케터야.
너는 항상 최선을 다하고 좋은 글을 작성해서 나를 기쁘게 해주고 있어.
아래의 형식으로 작성해줘.

#는 설명하는 말이니까 출력값에서 생략해줘
##는 꼭 필요한 형식이야
###는 단답식으로 작성해줘

{OUTPUT}을 채워주는거야

프롬프트에 의해서 출력값을 만들어 낼때는 아래의 조건을 만족해야 해.

1) 제목과 본문으로 구분해서 출력해줘.
2) 제목은 창의력 있고, 주목도 있게 구성해줘.
3) 본문을 구성할때는 Google과 Naver 검색의 검색엔진최적화(SEO)에 맞게 포스팅을 해줘
4) 최대한 자세하게 작성해주고 신뢰도 있는 정보를 중심으로 포스팅을 해줘
5) 글의 길이는 기본적으로 A4용지 1장 길이로 작성해줘
6) 간단 요약부분과 마지막말은 대학생 블로그 글 서포터즈가 쓴 글처럼 통통튀고 발랄한 말투로작성해줘
7) 공감하는 말을 추가해줘 ex) 지하철 파업으로 아침마다 출근하기 힘드네요

템플릿:



## {BLOG_TITLE}

## {INTRODUCTION} : 안녕하세요, 여러분의 출퇴근 메신저 지하철 온다의  '오.지.통 [오늘의 지하철 소식통]' 인사 드립니다!

#{START_DETAILS_SECTION}

-사건 일시:{OUTPUT}
-사건 지하철 노선: {OUTPUT}
-사건 발생 원인: {OUTPUT}
-문의사항:{RELATED_SITE_LINK}

#{END_DETAILS_SECTION}

## {SIMPLE_SUMMARY} : 위에서 작성한 Details 내용을 풀어서 요약

## {CLOSING_REMARKS} : {OUTPUT}

오지통이 실시간으로 다양한 지하철 정보를업데이트 할 예정이니, 자주 방문해 주세요.

'지하철 온다'는 단 한 번의 터치로

자신의 위치에서 가장 가까운 지하철역의

실시간 도착 정보를 제공합니다

🔽 지하철 온다 소개 보러가기

https://blog.naver.com/subway__onda/223258646349

입력문:
URL: {news_url}
"""

In [ ]:
# 평가 프롬프트(편집본)
consistency_prompt_2= """Given the BLOG POST and the NEWS ARTICLE, evaluate their consistency using the following scale and provide a score only:

1. 5 points: The BLOG POST logically follows from the NEWS ARTICLE.
2. 1 point: The BLOG POST contradicts the NEWS ARTICLE.
3. 2-4 points: Assign a score from 2 to 4 based on the degree of partial alignment or if determination of truth/falsehood requires more information.

Procedure:
- Extract key information from the NEWS ARTICLE.
- Determine if the BLOG POST's content is supported, contradicted, or partially aligned with the NEWS ARTICLE.
- Provide only the numerical score.

Notes:
- Ignore any symbols from the BLOG POST as they do not affect the score.
- Focus solely on content alignment for scoring.

Score:
- [Your score here based on the evaluation]"
"""

In [ ]:
# 편집한 이후의 평가 프롬프트 사용하는 걸로 함수 새로 정의

def final_func(df):
    # 결과 저장할 리스트 초기화
    answers_1 = []
    answers_2 = []

    # 데이터프레임 행들에 대한 반복문
    for index, row in df.iterrows():
        # 각 행의 URL에 맞게 프롬프트 내용 포매팅
        blog_prompt = prompt_template.format(
            news_url=row['URL'],
            OUTPUT='{OUTPUT}',
            BLOG_TITLE='{BLOG_TITLE}',
            INTRODUCTION='{INTRODUCTION}',
            START_DETAILS_SECTION='{START_DETAILS_SECTION}',
            EVENT_DATE='{EVENT_DATE}',
            EVENT_SUBWAY_LINE='{EVENT_SUBWAY_LINE}',
            EVENT_CAUSE='{EVENT_CAUSE}',
            CONTACT='{CONTACT}',
            RELATED_SITE_LINK='{RELATED_SITE_LINK}',
            END_DETAILS_SECTION='{END_DETAILS_SECTION}',
            SIMPLE_SUMMARY='{SIMPLE_SUMMARY}',
            CLOSING_REMARKS='{CLOSING_REMARKS}'
        )
        # GPT에 프롬프트 주입
        answer_1 = question_func(blog_prompt) # 블로그 작성 프롬프트
        answers_1.append(answer_1['text']) # 블로그 평가 프롬프트

        # 결과 리스트에 저장
        answer_2 = question_func(consistency_prompt_2)
        answers_2.append(answer_2['text'])

    # 데이터프레임의 새로운 열에 리스트 저장
    df['A1'] = answers_1 # 블로그 글
    df['A2'] = answers_2 # 평가 값

    return df # 새로운 데이터프레임 반환

In [ ]:
# news_test 에 함수 적용
final_func(news_test)

In [ ]:
# 결과 확인
news_test